# Limpieza de Datos Dataset Promociones-20190715

In [1]:
import pandas as pd
from utils_functions import convert_file, upload_to_bucket

#### Importar el dataset y observar los primeros registros con head y verificar los tipos de datos con el método info.

In [2]:
promo_col_names = [
        'idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'ventas_estimadas_und',
        'nro_prom', 'fecha_trans', 'porcentaje_dcto'
]

In [3]:
promo_df = pd.read_csv("../data/VMI_Promociones-20190715.txt",
                       encoding="utf-8",
                       header=None,
                       delimiter='\t',
                       names=promo_col_names)

In [4]:
promo_df.head()

,idb,id_producto,fecha_desde,fecha_hasta,ventas_estimadas_und,nro_prom,fecha_trans,porcentaje_dcto
0,242,228217,20161023,20161030,0.98,436830,20161005,7.0
1,242,228217,20161023,20161030,0.98,436830,20161006,7.0
2,242,228217,20161023,20161030,0.98,436830,20161007,7.0
3,242,228217,20161023,20161030,0.98,436830,20161008,7.0
4,242,228217,20161023,20161030,0.98,436830,20161009,7.0


In [5]:
promo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285253 entries, 0 to 285252
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   idb                   285253 non-null  int64  
 1   id_producto           285253 non-null  int64  
 2   fecha_desde           285253 non-null  int64  
 3   fecha_hasta           285253 non-null  int64  
 4   ventas_estimadas_und  285253 non-null  float64
 5   nro_prom              285253 non-null  int64  
 6   fecha_trans           285253 non-null  int64  
 7   porcentaje_dcto       285253 non-null  float64
dtypes: float64(2), int64(6)
memory usage: 17.4 MB


#### Ajustes al dataframe
- Convertir la columnas fecha_desde, fecha_hasta y fecha_trans a tipo de dato datetime.
- Verificar si existen valores nulos y/o repetidos.
- Obtener las fechas máximas y minimas de transaciones.
- Crear una nueva columnas llamada duracion_promo para obtener el delta de tiempo entre la fecha_desde y la fecha_hasta de una promoción.
- Obtener una análisis descriptivo de los datos.

In [6]:
promo_df = convert_file(promo_df, 'fecha_desde',  'date')

In [7]:
promo_df = convert_file(promo_df, 'fecha_hasta',  'date')

In [8]:
promo_df = convert_file(promo_df, 'fecha_trans', 'date')

In [9]:
promo_df.isnull().sum(axis=0)

idb                     0
id_producto             0
fecha_desde             0
fecha_hasta             0
ventas_estimadas_und    0
nro_prom                0
fecha_trans             0
porcentaje_dcto         0
dtype: int64

In [10]:
# Identificar los registros duplicados
duplicated_mask = promo_df.duplicated(subset=['idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'nro_prom'], keep=False)
duplicated_mask


0         True
1         True
2         True
3         True
4         True
          ... 
285248    True
285249    True
285250    True
285251    True
285252    True
Length: 285253, dtype: bool

In [11]:
# Filtrar los registros duplicados, manteniendo uno de ellos
promo_clean_df = promo_df[~duplicated_mask].append(promo_df[duplicated_mask].drop_duplicates(subset=['idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'nro_prom'], keep='first'))

/var/folders/1b/q0vy8pzj2d5_w7tb8dkfmh2m0000gn/T/ipykernel_82100/2594588663.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  promo_clean_df = promo_df[~duplicated_mask].append(promo_df[duplicated_mask].drop_duplicates(subset=['idb', 'id_producto', 'fecha_desde', 'fecha_hasta', 'nro_prom'], keep='first'))


***Valores duplicados***
Podemos ver que existen valores duplicados para cada item por ejemplo para el producto id 228217 tenemos 24 entradas las cuales contienen la misma fecha de desde y hasta, el mismo numero de promocion, para ver los demas valores de las columnas vamos a filtrar usando esos valores y ver que contiene las columnas cantidad.



In [12]:
promo_filtered = promo_clean_df[(promo_clean_df['fecha_desde'] == '2016-10-23') &
                                (promo_clean_df['fecha_hasta'] == '2016-10-30') &
                                (promo_clean_df['idb'] == 242) &
                                (promo_clean_df['id_producto'] == 228217)]
promo_filtered

,idb,id_producto,fecha_desde,fecha_hasta,ventas_estimadas_und,nro_prom,fecha_trans,porcentaje_dcto
0,242,228217,2016-10-23,2016-10-30,0.98,436830,2016-10-05,7.0


Podemos observar que 24 columnas son identicas con los mismo valores, la unica diferencia es la fecha_trans que causa que se dupliquen las filas, por lo tanto debemos filtrar el dataset para evitar estos datos duplicados.

Para limpiar el dataset y evitar valores duplicados vamos a eliminar los valores duplicados de filas que tienen identicos valores pero diferente fecha de transación, lo cual no tiene lógica porque este dataset contiene el registro de las promociones y no es un dataset de movimientos.



In [13]:
min_date = promo_df['fecha_desde'].min()
max_date = promo_df['fecha_hasta'].max()


print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Minimum date: 2016-07-19 00:00:00
Maximum date: 2019-07-16 00:00:00


In [14]:
min_date = promo_df['fecha_trans'].min()
max_date = promo_df['fecha_trans'].max()

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

Minimum date: 2016-07-14 00:00:00
Maximum date: 2019-07-14 00:00:00


In [15]:
# Crear una columna year, month and weekday from fecha_trans
promo_clean_df['year'] = promo_clean_df['fecha_trans'].dt.year
promo_clean_df['month'] = promo_clean_df['fecha_trans'].dt.month
promo_clean_df['weekday'] = promo_clean_df['fecha_trans'].dt.weekday

print(promo_clean_df['year'].value_counts(dropna=False))

2016    5347
2017    4261
2019    2896
2018    2766
Name: year, dtype: int64


In [16]:
promo_clean_df.describe()

,idb,id_producto,ventas_estimadas_und,nro_prom,porcentaje_dcto,year,month,weekday
count,15270.000000,15270.000000,15270.000000,15270.000000,15270.000000,15270.000000,15270.000000,15270.000000
mean,7187.241257,388184.750491,20.556557,472094.586182,8.320825,2017.210282,6.422921,3.009954
std,3128.103999,84684.306020,28.978951,38182.677713,2.885136,1.116144,2.969299,1.699944
min,242.000000,164146.000000,0.000000,425196.000000,3.000000,2016.000000,1.000000,0.000000
25%,5208.000000,342682.000000,3.120000,436830.000000,7.000000,2016.000000,4.000000,2.000000
50%,9036.000000,427445.000000,12.000000,456807.000000,7.000000,2017.000000,7.000000,3.000000
75%,9160.000000,458395.000000,24.560000,502704.000000,8.000000,2018.000000,8.000000,4.000000
max,9977.000000,507528.000000,300.820000,541066.000000,18.000000,2019.000000,12.000000,6.000000


In [17]:
promo_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15270 entries, 11133 to 285228
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   idb                   15270 non-null  int64         
 1   id_producto           15270 non-null  int64         
 2   fecha_desde           15270 non-null  datetime64[ns]
 3   fecha_hasta           15270 non-null  datetime64[ns]
 4   ventas_estimadas_und  15270 non-null  float64       
 5   nro_prom              15270 non-null  int64         
 6   fecha_trans           15270 non-null  datetime64[ns]
 7   porcentaje_dcto       15270 non-null  float64       
 8   year                  15270 non-null  int64         
 9   month                 15270 non-null  int64         
 10  weekday               15270 non-null  int64         
dtypes: datetime64[ns](3), float64(2), int64(6)
memory usage: 1.4 MB


In [18]:
promo_clean_df.head()

,idb,id_producto,fecha_desde,fecha_hasta,ventas_estimadas_und,nro_prom,fecha_trans,porcentaje_dcto,year,month,weekday
11133,277,273920,2017-01-15,2017-01-22,12.0,446149,2016-12-28,7.0,2016,12,2
11174,277,273921,2017-01-15,2017-01-22,12.0,446149,2016-12-28,7.0,2016,12,2
11175,277,284013,2017-01-15,2017-01-22,4.0,446149,2016-12-28,7.0,2016,12,2
12339,277,427444,2017-01-15,2017-01-22,12.0,446149,2016-12-28,7.0,2016,12,2
12451,277,427445,2017-01-15,2017-01-22,12.0,446149,2016-12-28,7.0,2016,12,2


#### Observaciones dataset Promociones-20190715

1. El dataset contiene la información referente a las promociones realizadas desde 2016-07-19 hasta el 2019-07-16.
2. El dataset contiene 285253 registros y 9 columnas descritas así:
    - idb:	Identificador interno de Boca
    - id_producto:	Identificador interno de Artículo
    - fecha-desde: 	Fecha desde de la vigencia de la Promoción
    - fecha_hasta:	Fecha hasta de la vigencia de la Promoción
    - ventas_estimadas_und: Estimado de ventas en la promoción en Unidades
    - nro_prom: Número de la Promoción
    - fecha_trans: Fecha de la Información
    - porcentaje_dcto: Porcentaje de Descuento
    - duracion_promo: La diferencia entre fecha_desde y fecha_hasta.
3. No se observan valores NAN ó duplicados.
4. La media de ventas estimadas es de 24.62 unidades.
5. El máximo porcentaje de descuento es de 18% y el mínimo es de 3%.
6. La duración máxima de una promoción es de 18 días.
7. La duración media de una promoción es de 8 días.

### Guardar el dataset limpio

In [19]:
promo_clean_df.to_csv('../clean_data/promociones_clean.csv', index=False)

#### Subir csv a google cloud storage

In [20]:
upload_to_bucket('deodorants', '../clean_data/promociones_clean.csv', 'promociones')

File ../clean_data/promociones_clean.csv uploaded successfully to bucket deodorants/promociones
